# crear una base de datos

### 1. crear un directorio llamado 16S

### 2. como se llama la base de datos

### 3. secuencias para la base de datos

#### preprocesamiento de la base de datos

#### eliminar microorganismos no cultivables
#### `python unculturedRemover.py Bacteria_RDP.fa bacteria_16S.fa`
https://raw.githubusercontent.com/Bioinformatica2020/Semana7/master/unculturedRemover.py

### 4. tipo de secuencia, <font color = red>prot</font>: para proteinas, <font color = red>nucl</font>: para nucleotidos

In [ ]:
makedb = subprocess.check_output(['makeblastdb','-in',fasta,'-dbtype',tipo,'-parse_seqids',
                 '-out', db])
print(makedb.decode())

# Clasificación taxonómica

#### Descarga del archivo

https://raw.githubusercontent.com/Bioinformatica2020/Semana7/master/16S_Seq.fasta

In [1]:
import random

### A. Seleccionar un número determinado de secuencias

#### Para el ejercicio en clase menos de 30

In [ ]:
FASTA = open('../descargas/16S_Seq.fasta','r')
FASTA = FASTA.read()

fas_dict = {}
for i, j in enumerate(FASTA.split('>')[1:]):
    fas_dict[i] = j
print('Número de secuencias:',len(re.findall('>', FASTA)))

secs = int(input('Número de secuencias a analizar (ejemplo: 50): '))
numero = random.sample(range(1, len(FASTA.split('>')[1:])), secs)
seleccionadas = [fas_dict[i] for i in numero]
query = '../salidas/'+str(secs)+'_secuencias.fasta'
cons = open(query,'w')
cons.write('>'+'>'.join(seleccionadas))
cons.close()
print('Archivo nuevo: ', query)

### B. Revisar los archivos dentro de la base de datos 16S

### C. Redefinir la base de datos, deacuerdo a los archivos mostrados en la celda anterior

### D. Correr blastn

In [ ]:
import subprocess
from datetime import datetime

In [ ]:
# https://www.ncbi.nlm.nih.gov/books/NBK279684/
xx = datetime.now()
subprocess.call([programa,'-db',db1,'-query', query,'-evalue',str(evalue),'-outfmt',
                                "6 qacc sacc qlen slen length qstart qend sstart send score bitscore evalue pident nident mismatch positive stitle",
                                '-max_target_seqs',str(10), '-max_hsps',str(10), '-out', query+'.tab'])

tim = datetime.now() - xx
print(format(tim).split('.')[0], '(h:m:s)')

### E. Explorar los resultados del Blastn

In [2]:
header = ('qacc','Entry','qlen','slen','length', 'qstart','qend','sstart','send','score','bitscore','evalue','pident','nident',
                              'mismatch','positive','stitle')

### F. A partir de la columna `stitle` obtener la clasificación taxonómica
#### Crea un DataFrame con esta información y llámalo: `Class_Tax`

## G. Visualización interactiva de los resultados  del Blastn

In [3]:
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt

In [ ]:
@interact
def scatter_plot(x=list(resultado.select_dtypes('number').columns), 
                 y=list(resultado.select_dtypes('number').columns),
                 siz = widgets.SelectionSlider(options=range(101),value=20,description='Point',
                                               disabled=False,
                              continuous_update=False,orientation='horizontal',readout=True),
                 alfa = widgets.SelectionSlider(options=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.9, 1],
                                                value=0.5,description='Alpha',disabled=False,
                              continuous_update=False,orientation='horizontal',readout=True),
                save = widgets.Checkbox(value=False, description='Save', disabled=False),
                xx = widgets.Text(value='Eje X',placeholder='Type something',description='axisX:',
    disabled=False),
                yy = widgets.Text(value='Eje Y',placeholder='Type something',description='axisY:',
    disabled=False)):
    plt.subplots(figsize=(5,5))
    #fig, ax = plt.subplots(figsize=(4,3))
    plt.scatter(resultado[x][resultado['pident'] >= 50], resultado[y][resultado['pident'] >= 50],
                s = siz, marker = 'o', c = 'blue', linewidth=0,
           edgecolors = 'white',alpha=alfa)
    plt.scatter(resultado[x][resultado['pident'] < 50], resultado[y][resultado['pident'] < 50],
                s = siz, marker = 'o', c = 'red', linewidth=0,
           edgecolors = 'white',alpha=alfa)
    plt.ylabel(yy,size=12,fontweight='bold')
    plt.xlabel(xx,size=12,fontweight='bold')
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)
    
    if save == True:
        plt.savefig('imgtesis/x_'+x+'_y_'+y+'.png',dpi = 900, bbox_inches='tight')
    if save == False:
        pass
    plt.show()

### H. Selecciona una categoría taxonómica para revisar la frecuencia

In [ ]:
# selecciona un elemento de la taxonomia
elemento = 'Species'
tx = blast_res[[elemento, 'Entry']].drop_duplicates().groupby([elemento]).Entry.count().reset_index()
tx[elemento] = ['<i>'+i+'</i>  ' for i in tx[elemento].tolist()]
tx = tx.sort_values(by="Entry", ascending = False)
tx

In [ ]:
import plotly.express as px

In [ ]:
fig = px.bar(tx, y=elemento, x="Entry",
             orientation="h",width=700, height=400)
fig.layout.plot_bgcolor = 'whitesmoke'
fig.update_yaxes(title_text='<b>Species</b>')
fig.update_xaxes(title_text='<b>Number of Entries (Entry)</b>')
fig.show()

## I. Visualización global de la clasificación taxonómica

In [ ]:
niveles = {'1':[0,7], '2':[0,1,7], '3':[0,1,2,7], '4':[0,1,2,3,7], '5':[0,1,2,3,4,7],
           '6':[0,1,2,3,4,5,7], '7':[0,1,2,3,4,5,6,7]}
colores = ['BrBG','PiYG','PRGn','PuOr','RdBu','RdGy','RdYlBu','RdYlGn','Spectral','Blues','BuGn','BuPu',
           'GnBu','Greens','Greys','Oranges','OrRd','PuBu','PuBuGn','PuRd','Purples','RdPu','Reds',
           'YlGn','YlGnBu','YlOrBr','YlOrRd']
colores.sort()

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Button, HBox, VBox, IntSlider, Label, IntRangeSlider, Layout
import datetime
import plotly.io

In [ ]:
@interact
def scatter_plot(Level = widgets.SelectionSlider(options=['1', '2', '3', '4', '5', '6', '7'],
                                                 value='6',description='Level',disabled=False,
                              continuous_update=False,orientation='horizontal',readout=True),
                 colors = widgets.Dropdown(options=colores,value='Oranges',description='Colors:',disabled=False,
                        layout=Layout(width='30%', height='30px')),
                 barescala = widgets.SelectionSlider(options=range(1,51),
                                                 value=5,description='Bar scale',disabled=False,
                              continuous_update=False,orientation='horizontal',readout=True),
                salvar = widgets.ToggleButton(value=False,description='Save html',disabled=False,button_style='',
                                tooltip='Description')):
    nivel = niveles[Level]
    df = Class_Tax.iloc[:,nivel]
    print(df.columns.tolist()[0:len(nivel)-1])
    fig = px.sunburst(Class_Tax.iloc[:,niveles[Level]], path=Class_Tax.iloc[:,niveles[Level]].columns.tolist()[0:len(niveles[Level])-1],
                      #title="Taxonomy Classification",
                      color='num', color_continuous_scale = colors, values='num')
    fig.update_layout(#uniformtext=dict(minsize=50),
        margin = dict(t=0, l=0, r=0, b=0),
        coloraxis_colorbar=dict(
    title="Abundance",
    thicknessmode="pixels",
    thickness=30,
    lenmode="pixels",
    #len=400,
    yanchor="bottom",
    xanchor="center",
    y=0,
    x=0.85,
    ticks="outside",
    dtick=barescala))
    fig.update_layout()
    fig.show()
    if salvar == True:
        plotly.offline.plot(fig, filename = archivo_salida.split('.')[0]+'_Tax_Class_'+datetime.datetime.now().strftime('%d.%B.%Y_%I-%M%p')+'.html', auto_open=False)

## Abrir otra tabla, con otra clasificacion

https://raw.githubusercontent.com/Bioinformatica2020/Semana7/master/Blastn_results.tab